In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
brands_automobile_dir_path = '/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/brands-dataset/automobile'
influencers_automobile_dir_path = '/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/influencer-dataset/automobile'

## Creating the Caption CSV for Brands

In [3]:
# Creating the Caption CSV for Brands

import csv

allXlsxfiles = [file for file in os.listdir('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/brands-dataset/automobile') if file.endswith('.xlsx')]

for file in allXlsxfiles:
    dr = os.path.join(brands_automobile_dir_path, file)

    brand_name = file.split('.')[0]

    df = pd.read_excel(dr)
    csv_file_path = os.path.join(brands_automobile_dir_path, f"{brand_name}.csv")
    with open(csv_file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['caption'])  # Write the header
        for caption in list(df['caption']):
            writer.writerow([caption])

## Creating the Caption CSV for Influencers

In [ ]:
# Creating the Caption CSV for Influencers

import csv

allXlsxfiles = [file for file in os.listdir(influencers_automobile_dir_path) if file.endswith('.xlsx')]

for file in allXlsxfiles:
    dr = os.path.join(influencers_automobile_dir_path, file)

    df = pd.read_excel(dr)

    tmp_df = df.groupby('ownerUsername')['caption'].agg(list).reset_index()

    folder_name = file.split('.')[0]
    folder_name = os.path.join('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/influencer-dataset/categories/automobile', folder_name)
    
    for index, row in tmp_df.iterrows():
        influencer = row['ownerUsername']
        captions = row['caption']

        csv_file_path = os.path.join(folder_name, f"{influencer}.csv")
        with open(csv_file_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(['caption'])  # Write the header
            for caption in captions:
                writer.writerow([caption])

In [234]:
alldr = os.listdir(influencers_automobile_dir_path)
alldr = [f for f in alldr if not f.endswith('.xlsx') and not f.endswith('.json') and not f.endswith('.DS_Store')]

brand_micro_influencer_list = []

for d in alldr:
    bpath = os.path.join(influencers_automobile_dir_path, d)
    lst = os.listdir(bpath)

    lst = [l for l in lst if not l.endswith('.csv')]

    for l in lst:
        newDir = {
            'brand': d,
            'micro-influencer': l,
            'ea': None
        }

        brand_micro_influencer_list.append(newDir)

In [235]:
brand_micro_influencer_list

bm_df = pd.DataFrame(brand_micro_influencer_list)
bm_df

,brand,micro-influencer,ea
0,volkswagen,thegtproduction,None
1,volkswagen,bandishprojekt,None
2,volkswagen,sidpatankar,None
3,volkswagen,kunalrawaldstress,None
4,volkswagen,sajad_machu,None
...,...,...,...
110,harleydavidson,apacheindianhq,None
111,harleydavidson,anushriyagulati,None
112,harleydavidson,broken.asphalt,None
113,harleydavidson,theweaero,None


In [3]:
import sys
sys.path.append('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/')
sys.path

['/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/rankings',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/.venv/lib/python3.11/site-packages',
 '/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/']

In [4]:
from embeddings.extract_visual_features import extract_and_flatten_features
from embeddings.extract_textual_features import extract_textual_features
from embeddings.weighted_pooling import weighted_history_pooling

In [5]:
import numpy as np

# Example usage
dv = 25088  # Dimensionality of visual features
dt = 300   # Dimensionality of textual features
dh1 = 300  # Dimensionality of the first hidden state vector
dh2 = 4096  # Dimensionality of the second hidden state vector
da = 512   # Dimensionality of the final social account representation
alpha = 0.5

def initialize_parameters(dv, dt, dh1, dh2, da):
    """
    Initialize parameters for the Multimodal Social Account Embedding.

    Parameters:
    - dv: Dimensionality of visual features.
    - dt: Dimensionality of textual features.
    - dh1: Dimensionality of the first hidden state vector.
    - dh2: Dimensionality of the second hidden state vector.
    - da: Dimensionality of the final social account representation.

    Returns:
    - Parameters Wt1, bt1, Wt2, Wv1, bv1, Wv2.
    """
    # Initialize parameters with uniform distribution between 0.1 and -0.1
    Wt1 = np.random.uniform(-0.1, 0.1, size=(dt, dh1))
    bt1 = np.random.uniform(-0.1, 0.1, size=(1, dh1))
    Wt2 = np.random.uniform(-0.1, 0.1, size=(dh1, da))
    
    Wv1 = np.random.uniform(-0.1, 0.1, size=(dv, dh2))
    bv1 = np.random.uniform(-0.1, 0.1, size=(1, dh2))
    Wv2 = np.random.uniform(-0.1, 0.1, size=(dh2, da))
    
    return Wt1, bt1, Wt2, Wv1, bv1, Wv2

In [6]:
# Initialize parameters
Wt1, bt1, Wt2, Wv1, bv1, Wv2 = initialize_parameters(dv, dt, dh1, dh2, da)

def non_linear_activation(x, alpha):
    """
    Apply Leaky ReLU activation function to the input.

    Parameters:
    - x: Input vector.
    - alpha: Slope for negative values (default is 0.01 for slight leakiness).

    Returns:
    - Output after applying the activation function.
    """
    return np.maximum(alpha * x, x)

def multimodal_social_account_embedding(ev, et) -> np.ndarray:
    """
    Perform Multimodal Social Account Embedding.

    Parameters:
    - ev: Visual features.
    - et: Textual features.
    - Wt1, bt1, Wt2, Wv1, bv1, Wv2: Parameters.

    Returns:
    - Final social account representation.
    """
    # Linear transformation followed by non-linear activation for textual features
    ht1 = non_linear_activation(np.dot(et, Wt1) + bt1, alpha)
    ht2 = np.dot(ht1[0], Wt2)
    
    # Linear transformation followed by non-linear activation for visual features
    hv1 = non_linear_activation(np.dot(ev, Wv1) + bv1, alpha)
    hv2 = np.dot(hv1[0], Wv2)
    
    # Concatenate and compute inner product to obtain final social account representation
    ea = np.multiply(ht2, hv2)
    
    return ea

## generating the final representation for the brands

In [57]:
alldr = os.listdir(brands_automobile_dir_path)
alldr = [f for f in alldr if not f.endswith('.xlsx') and not f.endswith('.json') and not f.endswith('.DS_Store') and not f.endswith('.csv')]

brand_list = []

for d in alldr:
    bpath = os.path.join(brands_automobile_dir_path, d)
    images = os.listdir(bpath)
    img = [os.path.join(bpath, i) for i in images]

    # -------------------------------------------------------------------- #
    # Image Part
    # -------------------------------------------------------------------- #
    
    # Extracting the visual features
    flattened_features = extract_and_flatten_features(img)
    ff = flattened_features

    # Applying the Weighted History Pooling method to the visual features
    ev = weighted_history_pooling(ff, 0.01)
    ev.reshape(-1, 1)
    # -------------------------------------------------------------------- #


    # -------------------------------------------------------------------- #
    # Textual Part
    # -------------------------------------------------------------------- #

    cp_df = pd.read_csv(f'{bpath}.csv')
    captions_list = list(cp_df['caption'])

    try:
        # Extracting the textual features
        textual_features = extract_textual_features(captions_list)

        # Applying the Weighted History Pooling method to the textual features
        et = weighted_history_pooling(textual_features, 0.01)
        et = et.reshape(-1, 1)

        # Updating the dataframe with visual and textual features
    except Exception as e:
        print(e)
    # -------------------------------------------------------------------- #

    ea = multimodal_social_account_embedding(ev.T, et.T)
    
    newDir = {
        'brand': d,
        'ea': ea
    }

    brand_list.append(newDir)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
bl_df = pd.DataFrame(brand_list)
bl_df.head()

In [59]:
bl_df.to_excel('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/Resources/brand_with_ea.xlsx', index=False)

## generating the final representation for the influencers

In [256]:
for index, rows in bm_df.iterrows():
    # print(rows['brand'])
    img_path = os.path.join(influencers_automobile_dir_path, rows['brand'] + '/' + rows['micro-influencer'])
    images = os.listdir(img_path)

    img = [os.path.join(img_path, i) for i in images]

    # -------------------------------------------------------------------- #
    # Image Part
    # -------------------------------------------------------------------- #
    
    # Extracting the visual features
    flattened_features = extract_and_flatten_features(img)
    ff = flattened_features

    # Applying the Weighted History Pooling method to the visual features
    ev = weighted_history_pooling(ff, 0.01)
    ev.reshape(-1, 1)

    # -------------------------------------------------------------------- #
    # Textual Part
    # -------------------------------------------------------------------- #

    cp_df = pd.read_csv(f'{img_path}.csv')
    captions_list = list(cp_df['caption'])

    try:
        # Extracting the textual features
        textual_features = extract_textual_features(captions_list)

        # Applying the Weighted History Pooling method to the textual features
        et = weighted_history_pooling(textual_features, 0.01)
        et = et.reshape(-1, 1)

        # Updating the dataframe with visual and textual features
    except Exception as e:
        print(img_path)
    
    ea = multimodal_social_account_embedding(ev.T, et.T)
    
    bm_df.at[index, 'ea'] = ea

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import pandas as pd
bl_df = pd.read_excel('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/Resources/brand_with_ea.xlsx')
bm_df = pd.read_excel('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/Resources/brand-micro-influencers_with_ea.xlsx')

bm_df.head()

,brand,micro-influencer,ea
0,volkswagen,thegtproduction,[ 1.80163463e+01 4.36932671e+00 -1.71532249e-...
1,volkswagen,bandishprojekt,[ 1.13089857e+01 1.75548182e+00 1.21754795e+...
2,volkswagen,sidpatankar,[ 2.56653011e+01 1.76183202e+01 1.14104894e+...
3,volkswagen,kunalrawaldstress,[ 6.49062687e+00 1.09081940e+01 3.90692331e-...
4,volkswagen,sajad_machu,[ 1.93117816e+01 -1.52468881e+00 1.40322390e+...


In [261]:
# bm_df.to_excel('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/Resources/brand-micro-influencers_with_ev_and_et.xlsx', index=False)

In [8]:
bm_df.groupby('brand').count()

,micro-influencer,ea
brand,,
ashokleyland,11,11
audiin,11,11
harleydavidson,8,8
hondacarindia,11,11
hyundaiindia,11,11
indiatriumph,11,11
jeepindia,11,11
mgmotorin,11,11
renault,10,10


### Calculating Similarity and engagement rate between brands and micro-influencers

In [9]:
print(bl_df.head())
print(bm_df.head())

           brand                                                 ea
0     volkswagen  [ 6.21427128e+00 -3.09665491e+00  1.49513928e-...
1        renault  [ 1.68256354e+01 -2.07551476e+01  1.12606369e+...
2   ashokleyland  [-1.75875926e+00 -2.22129716e+00 -2.40036121e+...
3  hondacarindia  [-1.68943254e+02 -2.59791729e+01 -1.50177738e+...
4   indiatriumph  [-2.23014285e-01 -2.95410740e+00 -3.23204507e+...
        brand   micro-influencer  \
0  volkswagen    thegtproduction   
1  volkswagen     bandishprojekt   
2  volkswagen        sidpatankar   
3  volkswagen  kunalrawaldstress   
4  volkswagen        sajad_machu   

                                                  ea  
0  [ 1.80163463e+01  4.36932671e+00 -1.71532249e-...  
1  [ 1.13089857e+01  1.75548182e+00  1.21754795e+...  
2  [ 2.56653011e+01  1.76183202e+01  1.14104894e+...  
3  [ 6.49062687e+00  1.09081940e+01  3.90692331e-...  
4  [ 1.93117816e+01 -1.52468881e+00  1.40322390e+...  


In [10]:
bl_df.rename(columns={'ea': 'brand_EA'}, inplace=True)
bm_df.rename(columns={'ea': 'influencers_EA'}, inplace=True)

In [11]:
# Join the DataFrames on the 'Brand' column
final_bm_df = pd.merge(bl_df, bm_df, on='brand')
final_bm_df.head()

,brand,brand_EA,micro-influencer,influencers_EA
0,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,thegtproduction,[ 1.80163463e+01 4.36932671e+00 -1.71532249e-...
1,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,bandishprojekt,[ 1.13089857e+01 1.75548182e+00 1.21754795e+...
2,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,sidpatankar,[ 2.56653011e+01 1.76183202e+01 1.14104894e+...
3,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,kunalrawaldstress,[ 6.49062687e+00 1.09081940e+01 3.90692331e-...
4,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,sajad_machu,[ 1.93117816e+01 -1.52468881e+00 1.40322390e+...


In [12]:
from rankings.calculate_competence_score import calculate_similarity

final_bm_df['sim'] = None

for index, rows in final_bm_df.iterrows():
    # Remove square brackets from the string
    brand_EA_str = rows['brand_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    brand_EA_list = [float(num) for num in brand_EA_str.split()]

    # Convert the list to a NumPy array
    brand_EA_array = np.array(brand_EA_list)

    # Remove square brackets from the string
    influencers_EA_str = rows['influencers_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    influencers_EA_list = [float(num) for num in influencers_EA_str.split()]

    # Convert the list to a NumPy array
    influencers_EA_array = np.array(influencers_EA_list)

    s = calculate_similarity(brand_EA_array, influencers_EA_array)
    
    final_bm_df.at[index, 'sim'] = s

In [13]:
final_bm_df.head()

,brand,brand_EA,micro-influencer,influencers_EA,sim
0,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,thegtproduction,[ 1.80163463e+01 4.36932671e+00 -1.71532249e-...,0.027137
1,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,bandishprojekt,[ 1.13089857e+01 1.75548182e+00 1.21754795e+...,0.041208
2,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,sidpatankar,[ 2.56653011e+01 1.76183202e+01 1.14104894e+...,0.035981
3,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,kunalrawaldstress,[ 6.49062687e+00 1.09081940e+01 3.90692331e-...,0.003633
4,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,sajad_machu,[ 1.93117816e+01 -1.52468881e+00 1.40322390e+...,0.021243


In [14]:
final_bm_df['engagement_rate'] = None

engrate_df = pd.read_csv('/Users/viru/Documents/GitHub/Micro-Influencer-Recommendations/Resources/engagement_rates.csv')

for index, rows in final_bm_df.iterrows():
    brand = rows['brand']
    influencer = rows['micro-influencer']

    # Corrected boolean indexing using the & operator and parentheses
    engagement_rate = engrate_df[(engrate_df['Brands'] == brand) & (engrate_df['Influencers'] == influencer)]['Engagement']
    
    # Check if any matching rows were found before assigning the value
    if not engagement_rate.empty:
        final_bm_df.at[index, 'engagement_rate'] = engagement_rate.iloc[0]


In [15]:
from rankings.calculate_competence_score import calculate_competence_score

# Define the trade-off between engagement_rate and similarity score
alpha = 0.5

# Iterate over each row in the DataFrame
for index, row in final_bm_df.iterrows():
    # Calculate the competence score using the formula
    engagement_score = row['engagement_rate']
    similarity_score = row['sim']  # Assuming you have a column named 'sim' for similarity
    
    # Check if both engagement_score and similarity_score are not NaN
    if not pd.isnull(engagement_score) and not pd.isnull(similarity_score):
        competence_score = calculate_competence_score(engagement_score, similarity_score, alpha)
        
        # Assign the competence score to a new column
        final_bm_df.at[index, 'competence_score'] = competence_score


In [16]:
final_bm_df.sort_values(by=['brand', 'competence_score'], ascending=[True, False], inplace=True)
final_bm_df

,brand,brand_EA,micro-influencer,influencers_EA,sim,engagement_rate,competence_score
24,ashokleyland,[-1.75875926e+00 -2.22129716e+00 -2.40036121e+...,lifeathgs,[ 1.30076832e+00 -5.06766180e+00 2.95137329e+...,0.021072,1.796859,0.908966
27,ashokleyland,[-1.75875926e+00 -2.22129716e+00 -2.40036121e+...,busfansadoor,[ 5.03881320e-01 2.77782385e+00 -4.39878379e+...,0.060342,1.529802,0.795072
25,ashokleyland,[-1.75875926e+00 -2.22129716e+00 -2.40036121e+...,_etauto,[ 1.19110854e+01 -5.05892363e+00 2.24003274e-...,-0.018258,0.486613,0.234177
21,ashokleyland,[-1.75875926e+00 -2.22129716e+00 -2.40036121e+...,8feet6wheels,[ 1.08912053e+01 1.41961205e+00 -1.32942186e-...,0.084529,0.203125,0.143827
28,ashokleyland,[-1.75875926e+00 -2.22129716e+00 -2.40036121e+...,gulfoil.india,[ 1.81257008e+01 3.46969697e+00 6.07767950e-...,0.054815,0.021289,0.038052
...,...,...,...,...,...,...,...
0,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,thegtproduction,[ 1.80163463e+01 4.36932671e+00 -1.71532249e-...,0.027137,0.031734,0.029435
3,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,kunalrawaldstress,[ 6.49062687e+00 1.09081940e+01 3.90692331e-...,0.003633,0.032482,0.018058
4,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,sajad_machu,[ 1.93117816e+01 -1.52468881e+00 1.40322390e+...,0.021243,0.00051,0.010876
6,volkswagen,[ 6.21427128e+00 -3.09665491e+00 1.49513928e-...,gti_lovers,[ 1.71555860e+01 1.08864549e+01 -1.90139810e+...,-0.02516,0.02589,0.000365


In [16]:
# Group by 'brand' and keep only the top 8 influencers for each group
final_bm_df = final_bm_df.groupby('brand').apply(lambda x: x.nlargest(8, 'competence_score')).reset_index(drop=True)

# final_bm_df['soft_cs'] = final_bm_df.groupby('brand')['competence_score'].transform(lambda x: np.exp(x) / np.exp(x).sum())

/var/folders/lp/thjl93cn1lg4b44b6kmtggvh0000gn/T/ipykernel_1160/3978209348.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_bm_df = final_bm_df.groupby('brand').apply(lambda x: x.nlargest(8, 'competence_score')).reset_index(drop=True)


### Training Part

In [17]:
test_set_brands = ['harleydavidson', 'renault']
train_set_brands = ['volkswagen', 'ashokleyland', 'hondacarindia', 
                    'indiatriumph', 'jeepindia', 'mgmotorin', 'audiin', 'hyundaiindia',
                    'skoda']

In [18]:
train_dataset = final_bm_df[final_bm_df['brand'].isin(train_set_brands)]
test_dataset = final_bm_df[final_bm_df['brand'].isin(test_set_brands)]
print("Train Set Shape:", train_dataset.shape)
print("Test Set Shape:", test_dataset.shape)

Train Set Shape: (72, 7)
Test Set Shape: (16, 7)


In [19]:
# Extract features and labels from the training dataset
# brand_representation_train = np.array(train_dataset['brand_EA'].tolist())
brand_representation_train = []
influencer_representation_train = []

for index, rows in train_dataset.iterrows():

    # Remove square brackets from the string
    brand_EA_str = rows['brand_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    brand_EA_list = [float(num) for num in brand_EA_str.split()]

    # Convert the list to a NumPy array
    brand_EA_array = np.array(brand_EA_list)

    brand_representation_train.append(brand_EA_array)

    # ----------------------------------------------------------------------

    # Remove square brackets from the string
    influencers_EA_str = rows['influencers_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    influencers_EA_list = [float(num) for num in influencers_EA_str.split()]

    # Convert the list to a NumPy array
    influencers_EA_array = np.array(influencers_EA_list)

    influencer_representation_train.append(influencers_EA_array)
    
similarity_train = np.array(train_dataset['sim'].tolist())
engagement_rate_train = np.array(train_dataset['engagement_rate'].tolist())
competence_score_train = np.array(train_dataset['competence_score'].tolist())

# print(type(influencer_representation_train[0]))


# Concatenate all the features into a single feature matrix
input_data_train = np.concatenate((brand_representation_train, 
                                    influencer_representation_train,
                                    similarity_train.reshape(-1, 1),
                                    engagement_rate_train.reshape(-1, 1)), axis=1)

In [20]:
x_train = input_data_train
print(x_train.shape)

y_train = competence_score_train.reshape(-1, 1)
print(y_train.shape)

(72, 1026)
(72, 1)


In [21]:
import tensorflow as tf

In [126]:
import tensorflow as tf

# Neural Network Architecture
class ListNetModel(tf.keras.Model):
    def __init__(self):
        super(ListNetModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu)
        self.dropout1 = tf.keras.layers.Dropout(0.5)  # Dropout layer with rate 0.5
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu)
        self.dropout2 = tf.keras.layers.Dropout(0.5)  # Dropout layer with rate 0.5
        self.dense_output = tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=tf.keras.regularizers.l1(0.001))  # L1 regularization with factor 0.001

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dropout1(x, training=training)  # Apply dropout during training
        x = self.dense2(x)
        x = self.dropout2(x, training=training)  # Apply dropout during training
        return self.dense_output(x)
    
# Initialize your model, optimizer, and metrics
model = ListNetModel()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean()

# ListWise Loss function
def listwise_loss_new(scores, competence_scores, lambda_reg=0.001):
    total_loss = 0
    num_brands = 9

    batch_size = 8
    no_of_batch = 9

    for batch in range(no_of_batch):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        
        batch_x = scores[start_idx:end_idx]  # Extract batch of input data
        batch_y = competence_scores[start_idx:end_idx]  # Extract batch of labels

        y_true = tf.cast(batch_y, tf.float32)
        y_pred = tf.cast(batch_x, tf.float32)

        y_true = tf.nn.softmax(y_true, axis=-1)
        y_pred = tf.nn.softmax(y_pred, axis=-1)

        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred + 1e-7))
        total_loss += loss

    total_loss /= num_brands

    # Add L1 regularization
    l1_norm = tf.reduce_sum([tf.reduce_sum(tf.abs(var)) for var in model.trainable_variables])
    total_loss += lambda_reg * l1_norm

    return total_loss

# Convert y_labels to sparse categorical
y_train_sparse = tf.squeeze(y_train, axis=-1)

# Define the training step function
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)  # Ensure training mode is enabled
        pred = tf.squeeze(predictions)
        
        loss = listwise_loss_new(labels, pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss.update_state(loss)  # Update the loss

# Training loop
epochs = 40
batch_size = 8
no_of_batch = len(train_dataset) // batch_size

for epoch in range(epochs):
    train_loss.reset_state()  # Reset the loss at the beginning of each epoch
    
    for batch in range(no_of_batch):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        
        batch_x = x_train[start_idx:end_idx]  # Extract batch of input data
        batch_y = y_train_sparse[start_idx:end_idx]  # Extract batch of labels

        train_step(batch_x, batch_y)  # Perform training step with the batch
        
    print(f'Epoch {epoch+1}, Loss: {train_loss.result()}')


Epoch 1, Loss: 10.326041221618652
Epoch 2, Loss: 8.252145767211914
Epoch 3, Loss: 6.492006301879883
Epoch 4, Loss: 4.945122718811035
Epoch 5, Loss: 3.760178565979004
Epoch 6, Loss: 2.7943522930145264
Epoch 7, Loss: 2.1096744537353516
Epoch 8, Loss: 1.728277325630188
Epoch 9, Loss: 1.4380512237548828
Epoch 10, Loss: 1.31161367893219
Epoch 11, Loss: 1.415352463722229
Epoch 12, Loss: 1.2353968620300293
Epoch 13, Loss: 1.1672824621200562
Epoch 14, Loss: 1.0467959642410278
Epoch 15, Loss: 0.9339534044265747
Epoch 16, Loss: 0.9198076725006104
Epoch 17, Loss: 0.9966742992401123
Epoch 18, Loss: 0.9293131232261658
Epoch 19, Loss: 0.8185895681381226
Epoch 20, Loss: 0.7581371068954468
Epoch 21, Loss: 0.7845619916915894
Epoch 22, Loss: 0.7780086994171143
Epoch 23, Loss: 0.8204657435417175
Epoch 24, Loss: 0.8044631481170654
Epoch 25, Loss: 0.7683126926422119
Epoch 26, Loss: 0.7159802317619324
Epoch 27, Loss: 0.663933277130127
Epoch 28, Loss: 0.6632474064826965
Epoch 29, Loss: 0.5967438220977783
Epo

In [128]:
tfdata = test_dataset[['brand_EA', 'influencers_EA', 'sim', 'engagement_rate', 'competence_score']]

brand_representation_train = []
influencer_representation_train = []

for index, rows in tfdata.iterrows():

    # Remove square brackets from the string
    brand_EA_str = rows['brand_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    brand_EA_list = [float(num) for num in brand_EA_str.split()]

    # Convert the list to a NumPy array
    brand_EA_array = np.array(brand_EA_list)

    brand_representation_train.append(brand_EA_array)

    # ----------------------------------------------------------------------

    # Remove square brackets from the string
    influencers_EA_str = rows['influencers_EA'].replace('[', '').replace(']', '')

    # Convert the string to a list of floats
    influencers_EA_list = [float(num) for num in influencers_EA_str.split()]

    # Convert the list to a NumPy array
    influencers_EA_array = np.array(influencers_EA_list)

    influencer_representation_train.append(influencers_EA_array)
    
similarity_train = np.array(tfdata['sim'].tolist())
engagement_rate_train = np.array(tfdata['engagement_rate'].tolist())
competence_score_train = np.array(tfdata['competence_score'].tolist())

# Concatenate all the features into a single feature matrix
input_data_train_test = np.concatenate((brand_representation_train, 
                                    influencer_representation_train,
                                    similarity_train.reshape(-1, 1),
                                    engagement_rate_train.reshape(-1, 1)), axis=1)

test_prediction = model.predict(input_data_train_test)
actual_label = competence_score_train

test_prediction = tf.squeeze(test_prediction)
print(actual_label)
print(test_prediction)
# mse = tf.keras.metrics.MeanSquaredError()
# print(mse.update_state(actual_label, test_prediction))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[1.61135149 0.1945841  0.07202088 0.03597747 0.02404899 0.01673853
 0.01228976 0.01137302 0.12550586 0.09854912 0.06078267 0.04915757
 0.03426271 0.03171024 0.03026108 0.02689485]
tf.Tensor(
[-2.3223509e-01  7.1584487e+00  1.5667354e+01  1.2765273e+01
  8.6937332e+00  8.2921310e+06  6.7891564e+00  7.1555777e+00
  1.2533097e+01  4.1787586e+00  8.6861696e+00  1.0205772e+01
  6.1731668e+00  7.1673112e+00  1.2832602e+01  8.2358561e+00], shape=(16,), dtype=float32)


In [129]:
print(competence_score_train)

[1.61135149 0.1945841  0.07202088 0.03597747 0.02404899 0.01673853
 0.01228976 0.01137302 0.12550586 0.09854912 0.06078267 0.04915757
 0.03426271 0.03171024 0.03026108 0.02689485]
